In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported successfully.')

Libraries imported successfully.


# session 3

In [3]:

df_merged = pd.read_csv('Merged_Data.csv').iloc[:, 1:]
df_merged.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:

#Will be used later to merge dataframes
df_later = df_merged.loc[:, ['Borough', 'Neighborhood']]
df_later.head()

,Borough,Neighborhood
0,Scarborough,"Rouge, Malvern"
1,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,Scarborough,"Guildwood, Morningside, West Hill"
3,Scarborough,Woburn
4,Scarborough,Cedarbrae


In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add markers to Toronto map
for lat, lng, postalcode, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'],df_merged['Postal Code'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}, {}'.format(postalcode, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#e334c3',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Exploring Various Places in Neighborhoods

In [7]:
#Initialising Foursquare Credentials
CLIENT_ID = 'BAMVZJNDJFCXQ3KDKHU1TFVEH3ERVHOSW4X1EL4H50BUWKRN' # your Foursquare ID
CLIENT_SECRET = 'GZ0W1BD4KNYN4FNURB2QB5BT0YWUB4ZNOYYB5GH5MQGG5XTW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
limit = 5

Your credentials:
CLIENT_ID: BAMVZJNDJFCXQ3KDKHU1TFVEH3ERVHOSW4X1EL4H50BUWKRN
CLIENT_SECRET:GZ0W1BD4KNYN4FNURB2QB5BT0YWUB4ZNOYYB5GH5MQGG5XTW


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
     
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [10]:
toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

In [12]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[99]] + list(toronto_onehot.columns[:98]) + list(toronto_onehot.columns[100:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Organic Grocery,Airport,Airport Food Court,Airport Gate,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Neighborhood,Other Repair Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Grouping the one hot dataframe by Borough


In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_merged = df_later.merge(toronto_grouped, on='Neighborhood')
toronto_merged_drop = toronto_merged.drop('Neighborhood', axis=1)
toronto_grouped_borough = toronto_merged_drop.groupby('Borough').mean().reset_index()
toronto_grouped_borough.head()

,Borough,Organic Grocery,Airport,Airport Food Court,Airport Gate,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Other Repair Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.044444,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.022222,0.00,0.0,0.000000,0.00,0.00,0.037037,0.0,0.00,0.0,0.044444,0.0,0.022222,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.022222,0.044444,0.022222,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.022222,0.0,0.00,0.0,0.111111,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.022222,0.00,0.044444,0.0,0.000000,0.000000,0.027778,0.0,0.0,0.022222,0.0,0.0,0.0,0.0,0.0,0.022222,0.022222,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.087037,0.00,0.00,0.022222,0.037037,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.022222,0.00,0.000000,0.022222,0.000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.00,0.0,0.037037,0.022222,0.00,0.05,0.037037,0.000000,0.000000,0.000000,0.0,0.064815,0.022222,0.0,0.00,0.00,0.0,0.022222
1,Downtown Toronto,0.010526,0.010526,0.010526,0.010526,0.010526,0.010526,0.010526,0.0,0.0,0.021053,0.00,0.0,0.000000,0.0,0.010526,0.00,0.00,0.0,0.010526,0.00,0.0,0.010526,0.00,0.00,0.000000,0.0,0.00,0.0,0.042105,0.0,0.000000,0.010526,0.115789,0.0,0.010526,0.021053,0.000000,0.000000,0.00,0.0,0.00,0.010526,0.010526,0.000000,0.010526,0.021053,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.00,0.010526,0.052632,0.010526,0.0,0.010526,0.00,0.0,0.0,0.0,0.021053,0.00,0.010526,0.0,0.052632,0.031579,0.000000,0.0,0.0,0.010526,0.0,0.0,0.0,0.0,0.0,0.021053,0.000000,0.000,0.0,0.0,0.0,0.021053,0.0,0.0,0.068421,0.00,0.00,0.010526,0.013158,0.021053,0.0,0.010526,0.021053,0.010526,0.00,0.094737,0.00,0.010526,0.000000,0.000,0.0,0.00,0.0,0.010526,0.0,0.010526,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.010526,0.010526,0.010526,0.0,0.013158,0.031579,0.0,0.00,0.00,0.0,0.000000
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.04,0.0,0.000000,0.04,0.0,0.000000,0.00,0

In [15]:
num_top_venues = 5

for bor in toronto_grouped_borough['Borough']:
    print("----"+bor+"----")
    temp = toronto_grouped_borough[toronto_grouped_borough['Borough'] == bor].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Central Toronto----
              venue  freq
0            Garden  0.11
1              Park  0.09
2             Trail  0.06
3  Sushi Restaurant  0.05
4        Playground  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.12
1          Restaurant  0.09
2                Park  0.07
3                 Gym  0.05
4  Italian Restaurant  0.05


----East Toronto----
               venue  freq
0     Ice Cream Shop  0.12
1   Greek Restaurant  0.08
2  Health Food Store  0.05
3              Trail  0.05
4                Pub  0.05


----East York----
               venue  freq
0           Pharmacy  0.12
1                Gym  0.08
2  Convenience Store  0.07
3               Park  0.07
4        Coffee Shop  0.07


----Etobicoke----
            venue  freq
0     Pizza Place  0.12
1  Sandwich Place  0.06
2     Coffee Shop  0.06
3            Park  0.05
4       Drugstore  0.05


----Mississauga----
             venue  freq
0            Hotel   0.4
1    Burrito Place   0

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Borough'] = toronto_grouped_borough['Borough']

for ind in np.arange(toronto_grouped_borough.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_borough.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Garden,Park,Trail,Sushi Restaurant,Indian Restaurant,American Restaurant,Dessert Shop,Café,Bus Line,Swim School
1,Downtown Toronto,Coffee Shop,Restaurant,Park,Italian Restaurant,Gym,Café,Vegetarian / Vegan Restaurant,Japanese Restaurant,Concert Hall,Plaza
2,East Toronto,Ice Cream Shop,Greek Restaurant,Trail,Pub,Health Food Store,Coffee Shop,Farmers Market,Park,Pet Store,Cosmetics Shop
3,East York,Pharmacy,Gym,Park,Coffee Shop,Convenience Store,Yoga Studio,Pizza Place,Liquor Store,Indian Restaurant,Fish & Chips Shop
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Park,Rental Car Location,River,Drugstore,Bakery,Liquor Store,Beer Store
5,Mississauga,Hotel,Coffee Shop,Sandwich Place,Burrito Place,Yoga Studio,Dessert Shop,Drugstore,Dog Run,Discount Store,Diner
6,North York,Park,Coffee Shop,Cafeteria,Grocery Store,Café,Bank,Baseball Field,Pizza Place,Bakery,Japanese Restaurant
7,Scarborough,Playground,Coffee Shop,Fast Food Restaurant,Bar,Skating Rink,Chinese Restaurant,Breakfast Spot,Sandwich Place,Thai Restaurant,Indian Restaurant
8,West Toronto,Coffee Shop,Bar,Brewery,Italian Restaurant,Cuban Restaurant,Park,Gastropub,Pizza Place,Middle Eastern Restaurant,Pub
9,York,Park,Convenience Store,Sandwich Place,Grocery Store,Women's Store,Trail,Skating Rink,Brewery,Hockey Arena,Market


# Code for Sorting by Neighborhood

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [19]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters